In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import pickle
import os
import numpy as np
from collections import defaultdict
from tqdm import tqdm

In [17]:
i = 0
x = []
y = []

dir = os.path.expanduser('~/Desktop/inception_activations')
for filename in tqdm(sorted(os.listdir(dir))):
    if filename.endswith('.pickle'):
        with open(os.path.join(dir, filename)) as f:
            for label, vector in pickle.load(f).iteritems():
                x.append(vector)
                y.append(i)
        i += 1

x = np.stack(x)
y = np.array(y, dtype=np.int32)

n_classes = np.max(y)
input_dim = x.shape[-1]
print input_dim, '->', n_classes


100%|██████████| 65/65 [00:20<00:00,  3.02it/s]


In [28]:
print x.shape, np.expand_dims(y, 1).shape

(65000, 2048) (65000, 1)


In [29]:
model = Sequential()
model.add(Dense(1024, input_shape=(input_dim,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(n_classes))
model.add(Activation('softmax'))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
activation_7 (Activation)    (None, 1024)              0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                65600     
_________________________________________________________________
activation_8 (Activation)    (None, 64)                0         
Total params: 2,163,776.0
Trainable params: 2,163,776.0
Non-trainable params: 0.0
_________________________________________________________________
None


In [ ]:
model.fit(x, np.expand_dims(y, 1), epochs=10, batch_size=32, validation_split=0.2)


Train on 52000 samples, validate on 13000 samples
Epoch 1/10
  128/52000 [..............................] - ETA: 386s - loss: 15.1168 - acc: 0.0312 